In [1]:
# Load packages files
from dotenv import load_dotenv
import os
import base64 
from requests import post, get
import json


In [3]:
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    # Retrieve authurization token
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [5]:
# Retrieve authenication token
# print(get_token())

In [6]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    # pass in the requested query
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query 
    result = get(query_url, headers=headers)

    # Parse results to json
    json_result = json.loads(result.content)["artists"]["items"]

    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    
    return json_result[0]

In [7]:
def get_songs_by_artist(token, artist_id):
    # Create a functions to get songs from an artist
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

token = get_token()
result = search_for_artist(token, "kirk+franklin")
artist_id = result["id"]
songs = get_songs_by_artist(token, artist_id)

artist_name = result["name"]
song_genre = result["genres"]

list_as_string = ' | '.join(map(str, song_genre))
print(f'Artist name: {artist_name} \nGenres: {list_as_string} \nTop 10 songs:\n')

for idx, song in enumerate(songs):
    print(f"{idx +1}. {song['name']}")


Artist name: Kirk Franklin 
Genres: gospel | gospel r&b 
Top 10 songs:

1. TOGETHER
2. Love Theory
3. I Smile
4. Kingdom
5. Bless Me
6. Fear Is Not My Future (feat. Brandon Lake)
7. Brighter Day - Live at Lakewood Church, Houston, TX - June 16, 2000
8. Imagine Me
9. All Things
10. Revolution
